In [1]:
import pandas as pd
from faker import Faker
import random
import sys
sys.path.append('..')
from app.models.suggestions import SuggestionManager

# Initialize SuggestionManager
suggestion_manager = SuggestionManager()

# Load or generate data
try:
    names_df = pd.read_csv('../data/raw/NationalNames.csv')
    names = names_df['Name'].unique().tolist()[:5000]  # Limit for simplicity
except FileNotFoundError:
    print("Download 'NationalNames.csv' from https://www.kaggle.com/datasets/kaggle/us-baby-names")
    fake = Faker()
    names = [fake.name() for _ in range(5000)]

# Generate emails
fake = Faker()
emails = [fake.email() for _ in range(5000)]

# Combine and save dataset
dataset = names + emails
random.shuffle(dataset)
df = pd.DataFrame(dataset, columns=['text'])
df.to_csv('../data/processed/autocomplete_dataset.csv', index=False)
print(f"Dataset size: {len(dataset)}")
df.head()

# Add to SuggestionManager
for text in dataset:
    category = 'email' if '@' in text else 'name'
    suggestion_manager.add_suggestion(text, category)

# Save trie
suggestion_manager.save_trie()
print("Trie model trained and saved.")

# Test model
print("Testing prefix 'jo':")
print(suggestion_manager.get_suggestions('jo', limit=5))
print("\\nTesting prefix 'test@':")
print(suggestion_manager.get_suggestions('test@', limit=5))

UnpicklingError: pickle data was truncated